In [1]:
ls_columns_result = [
    'Врем. сопротивление',
    'прогноз Врем. сопротивление',
    'MAE Врем. сопротивление',
    'MSE Врем. сопротивление',
    'Предел текучести',
    'прогноз Предел текучести', 
    'MAE Предел текучести',
    'MSE Предел текучести',
]
ls_columns_output = [
    '№ партии',
    'марка стали',
    'диаметр',
    'толщина стенки',
    '1 зона по ВТР закалка',
    '2 зона по ВТР закалка',
    '3 зона по ВТР закалка',
    'шаг балок закалочная печь, сек',
    'Скорость прохождения трубы через спрейер, м/с', 
    't˚ C трубы после спреера',
    '1 зона ВТР и уставка отпуск', 
    '2 зона ВТР и уставка отпуск', 
    '3 зона ВТР и уставка отпуск',
    '4 зона ВТР и уставка отпуск',
    '5 зона ВТР и уставка отпуск',
    'шаг балок отпускная печь, сек',
    'C',
    'Mn',
    'Si',
    'P',
    'S',
    'Cr',
    'Ni',
    'Cu',
    'Al',
    'V',
    'Ti',
    'Nb',
    'Mo',
    'N',
    'B',
    'C-coef',
    'Параметр закалка',
    'Параметр отпуск',
    'Параметр отпуск новый V',
    'Величина зерна',
    'Тип предела текучести (1186)',
    'ICD'
    ]

In [2]:
import pandas as pd
import numpy as np
import pickle
import json
import import_ipynb
from my_libs.calc_features import *
from sklearn.preprocessing import StandardScaler
from datetime import datetime


importing Jupyter notebook from D:\Repo\PycharmProjects\gitPNTZ\PNTZ\my_libs\calc_features.ipynb


In [28]:
load = datetime.now()
database = pd.read_excel('prepared_to_saw.xlsx')
print('База исторических режимов загружена за ', datetime.now() - load)

cleaned_input = pd.DataFrame()

def close_value(df, row, value):
    s_close = df[row].value_counts().index[0]
    min_delta = np.abs(s_close-value)
    for s_ in df[row].value_counts().index:
        delta = np.abs(s_-value)
        if delta<min_delta:
            min_delta = delta
            s_close = s_
    return s_close

def find_close_sort(df):
    tmp = pd.DataFrame()
    for i in range(len(df.index)):
        answ = df.iloc[i, :].copy()
        mark = answ['марка стали']
        d = answ['диаметр']
        s = answ['толщина стенки']
        if database[database['марка стали']==mark].shape[0]==0:
            mark = mark.split('-')[0]
            answ = database[database['марка стали']==mark]
        if database[database['диаметр']==d].shape[0]!=0:
            answ = database[database['диаметр']==d]
            s_close = close_value(database, 'толщина стенки',s)
            answ = database[database['толщина стенки']==s_close]
        elif database[database['толщина стенки']==s].shape[0]!=0:
            answ = database[database['толщина стенки']==s]
            d_close = close_value(database, 'диаметр',d)
            answ = database[database['диаметр']==d_close]
        answ = answ[answ['Дата термообработки'] == answ['Дата термообработки'].max()][:1]
#         print('Исторические режимы найдены', answ.shape)
        tmp = pd.concat([tmp, answ])
    return tmp 

def find_mark_typesize(df):

    for i in range(len(df.index)):
        answ = df.iloc[i, :].copy()
        mark = answ['марка стали']
        d = answ['диаметр']
        s = answ['толщина стенки']
        if mark!=None:
            answ = database[database['марка стали']==mark]
        if d!=None:
            answ = database[database['диаметр']==d]
        if s!=None:
            answ = database[database['толщина стенки']==s]
        answ = answ[answ['Дата термообработки'] == answ['Дата термообработки'].max()][:3]
#         print('Исторические режимы найдены', answ.shape)
        tmp = pd.concat([tmp, answ])
    return tmp

# возвращает модель, список признаков, по которому строилась модель и скейлер
def load_model(dir_name, target):
    model = pickle.load(open(dir_name+'/RF_model_'+target+'.sav', 'rb'))
    ls_need_col = json.load(open(dir_name+'/ls_need_col', "r"))
#     try:
    scaler = StandardScaler()
    scale_data = json.load(open(dir_name+'/scaler', "r"))
    scaler.mean_ = scale_data[0]
    scaler.scale_ = scale_data[1]
#     except:
#         scaler=None
    return model, ls_need_col, scaler


def make_result_valid_file(file_name, dir_names, targets, output_filename):
    """Сохраняет файл с результатами """
    result = pd.DataFrame()
    base = pd.DataFrame()
    k = 0
    for target, dir_name in zip(targets,dir_names):
        model, ls_need_col, scaler = load_model(dir_name, target)
        time = datetime.now()
        if k == 0:
            try:
                valid = clean_data(file_name, ls_columns_output)
            except KeyError:
                valid = clean_data(file_name, list(set(ls_columns_output) & set(['№ партии', "марка стали"])))
            cleaned_input = valid.copy()
            print('Введенные данные очищены за ', datetime.now() - time)
        else:
            valid = cleaned_input.copy()
    #     print(valid['C-coef'])
    #     valid = valid[valid['C-coef']<0.6]
        time = datetime.now()
        try:
            y_valid = valid[target].copy()
        except KeyError:
            y_valid = [0 for x in range(0,valid.shape[0])]
        try:
            X_valid = valid[ls_columns_output].copy()
        except KeyError:
            X_valid = valid[ls_need_col].copy()
        print('Магия try-except`ов за ', datetime.now() - time)
        X_valid_c = X_valid[ls_need_col].copy()
        X_valid_c = scaler.transform(X_valid_c)
        time = datetime.now()
        y_pred = model.predict(X_valid_c)
        print('Предсказания ', y_pred.shape[0], ' строчек за ', datetime.now() - time)
        result[target] = y_valid
        result['прогноз '+target] = y_pred
        result['MAE '+target] = np.abs(result[target]- result['прогноз '+target])
        result['MSE '+target] = result['MAE '+target].apply(lambda x: x*x)
        k += 1

#     result['отношение'] = result[targets[0]]/result[targets[1]]
    time = datetime.now()
    base = find_close_sort(X_valid)
    print('Найдены базовые исторические режимы за ', datetime.now() - time)
    time = datetime.now()
    result = pd.concat([result, X_valid], axis=1)
    print('Конкатенация прогноза и введенных режимов за ', datetime.now() - time)
    time = datetime.now()
    base = base[list(set(base.columns) & set(result.columns))]
    base.index = [str(i)+' исторический' for i in range(len(base.index))]
    print('Преобразование базовых режимов за ', datetime.now() - time)
    con = pd.DataFrame()
    time = datetime.now()
    missing = ['' for i in result.index]
    missing = pd.DataFrame(missing)
#     result.index = [str(i) for i in range(len(result.index))]
#     con = pd.concat([result, base])
    for i, j, k in zip(result.index, base.index, missing.index):
        con = pd.concat([con, result[result.index == i], base[base.index == j], missing[missing.index == k]])
    print('Конкатенация базовых и введенных режимов за ', datetime.now() - time)
    con = con[ls_columns_result + ls_columns_output]
    con.to_excel(output_filename)
    return con, y_valid, y_pred, base

def make_result_valid_file_multi(file_name, dir_names, targets, output_filename):
    """Сохраняет файл с результатами """
    writer = pd.ExcelWriter(output_filename)
    df_dict = pd.read_excel(file_name, sheetname=None)
    for sheet, df in zip(df_dict, df_dict.values()):
        result = pd.DataFrame()
        base = pd.DataFrame()
        k=0
        for target, dir_name in zip(targets,dir_names):
            model, ls_need_col, scaler = load_model(dir_name, target)
            valid = clean_data_sheets(df, ls_need_col+['№ партии', 'марка стали'])
            y_valid = valid[target].copy()
            X_valid = valid[ls_columns_output].copy()
            X_valid_c = X_valid[ls_need_col].copy()

            y_pred = model.predict(X_valid_c)

            result[target] = y_valid
            result['прогноз '+target] = y_pred
            result['MAE '+target] = np.abs(result[target]- result['прогноз '+target])
            result['MSE '+target] = result['MAE '+target].apply(lambda x: x*x)
            X_valid = new_spr(X_valid)
            k+=1
            if k==2:
                result = pd.concat([result, X_valid], axis=1)
#                 result = pd.concat([result, base])
                result.to_excel(writer, sheet)
                
    writer.save()
    return result, y_valid, y_pred, base

file_name = 'D:\Repo\PycharmProjects\RELEASE\PNTZ_sol_yuk\PNTZ_sol_yuk\input\input.xlsx'
# target = 'Врем. сопротивление'
targets = ['Предел текучести','Врем. сопротивление']
dir_names = ['DATA/MODELS_RF/YS RF valid', 'DATA/MODELS_RF/H RF valid']
output_filename = 'DATA/modes_after_model/127x12,5 37g2f _ input - копия test find base modes.xlsx'

time = datetime.now()
df, y_valid, y_pred, dbasemode = make_result_valid_file(file_name, dir_names, targets, output_filename)
print('Время работы скрипта на ', df.shape[0]/2, 'строчках: ', datetime.now() - time)

База исторических режимов загружена за  0:00:17.898024


D:\Repo\PycharmProjects\gitPNTZ\PNTZ\my_libs\calc_features.ipynb:83: RuntimeWarning: divide by zero encountered in log10
  "    print(df.shape)\n",
D:\Repo\PycharmProjects\gitPNTZ\PNTZ\my_libs\calc_features.ipynb:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  {


(1778, 40)
(1516, 40)
Введенные данные очищены за  0:01:18.054464
Магия try-except`ов за  0:00:00.002000
Предсказания  1516  строчек за  0:00:00.005000
Магия try-except`ов за  0:00:00.001001
Предсказания  1516  строчек за  0:00:00.006000
Найдены базовые исторические режимы за  0:00:38.044176
Конкатенация прогноза и введенных режимов за  0:00:00.002000
Преобразование базовых режимов за  0:00:00.003001


D:\Repo\Anaconda2\envs\myenv\lib\site-packages\pandas\core\indexes\api.py:57: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  union = _union_indexes(indexes)
D:\Repo\Anaconda2\envs\myenv\lib\site-packages\pandas\core\indexes\api.py:87: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  result = result.union(other)


Конкатенация базовых и введенных режимов за  0:00:14.513830
Время работы скрипта на  2274.0 строчках:  0:02:15.188732


In [27]:
missing = ['' for i in df.index]
missing = pd.DataFrame(missing)
pd.concat([df, missing])

D:\Repo\Anaconda2\envs\myenv\lib\site-packages\pandas\core\indexes\api.py:87: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)


,Врем. сопротивление,прогноз Врем. сопротивление,MAE Врем. сопротивление,MSE Врем. сопротивление,Предел текучести,прогноз Предел текучести,MAE Предел текучести,MSE Предел текучести,№ партии,марка стали,...,N,B,C-coef,Параметр закалка,Параметр отпуск,Параметр отпуск новый V,Величина зерна,Тип предела текучести (1186),ICD,0
0,77.244898,97.891224,20.646327,426.270799,65.943878,96.444808,30.500930,930.306731,803335,30ХГМА,...,0.00700,0.0,0.632333,875.055055,18.655439,10.725832,25.304650,0.5,2.990870,NaN
1,62.375000,73.114702,10.739702,115.341207,50.875000,58.852348,7.977348,63.638082,804315,13ХФА,...,0.00620,0.0,0.368038,913.071644,19.045678,13.206535,17.193802,0.2,0.672482,NaN
2,77.959184,86.444099,8.484915,71.993782,71.683673,95.170168,23.486494,551.615402,804306,18Х3МФБ,...,0.00712,0.0,0.839606,874.484660,18.300459,9.617827,5.400362,0.5,3.583364,NaN
3,81.760204,86.444099,4.683895,21.938868,70.612245,95.170168,24.557923,603.091563,804307,18Х3МФБ,...,0.00712,0.0,0.839606,874.484660,18.300459,9.617827,5.400362,0.5,3.583364,NaN
4,79.668367,86.444099,6.775731,45.910535,72.729592,95.170168,22.440576,503.579436,804308,18Х3МФБ,...,0.00712,0.0,0.839606,874.484660,18.300459,9.617827,5.400362,0.5,3.583364,NaN
5,83.545918,86.444099,2.898180,8.399449,73.469388,95.170168,21.700780,470.923842,804312,18Х3МФБ,...,0.00712,0.0,0.839606,874.484660,18.300459,9.617827,5.400362,0.5,3.583364,NaN
6,79.744898,86.444099,6.699201,44.879290,72.653061,95.170168,22.517106,507.020075,804302,18Х3МФБ,...,0.00712,0.0,0.839606,874.330783,18.300459,9.617827,5.396623,0.5,3.583364,NaN
7,75.714286,86.444099,10.729813,115.128885,69.132653,95.170168,26.037514,677.952159,804304,18Х3МФБ,...,0.00712,0.0,0.839606,874.484660,18.300459,9.617827,5.400362,0.5,3.583364,NaN
8,79.974490,86.444099,6.469609,41.855839,71.428571,95.170168,23.741596,563.663384,804309,18Х3МФБ,...,0.00712,0.0,0.839606,874.484660,18.300459,9.617827,5.400362,0.5,3.583364,NaN
9,81.734694,86.444099,4.709405,22.178493,72.959184,95.170168,22.210984,493.327803,804310,18Х3МФБ,...,0.00712,0.0,0.839606,874.484660,18.300459,9.617827,5.400362,0.5,3.583364,NaN


In [22]:
df.sort_index()

,Врем. сопротивление,прогноз Врем. сопротивление,MAE Врем. сопротивление,MSE Врем. сопротивление,Предел текучести,прогноз Предел текучести,MAE Предел текучести,MSE Предел текучести,№ партии,марка стали,...,Mo,N,B,C-coef,Параметр закалка,Параметр отпуск,Параметр отпуск новый V,Величина зерна,Тип предела текучести (1186),ICD
0,77.244898,97.891224,20.646327,426.270799,65.943878,96.444808,30.500930,930.306731,803335,30ХГМА,...,0.15000,0.00700,0.0,0.632333,875.055055,18.655439,10.725832,25.304650,0.5,2.990870
0 исторический,77.244898,NaN,NaN,NaN,65.943878,NaN,NaN,NaN,803335,30ХГМА,...,0.15000,0.00700,NaN,0.632333,875.055055,18.655439,10.725832,25.304650,0.5,2.990870
1,62.375000,73.114702,10.739702,115.341207,50.875000,58.852348,7.977348,63.638082,804315,13ХФА,...,0.00698,0.00620,0.0,0.368038,913.071644,19.045678,13.206535,17.193802,0.2,0.672482
1 исторический,80.994898,NaN,NaN,NaN,72.806122,NaN,NaN,NaN,804084,18Х3МФБ,...,0.11000,0.00900,NaN,0.898667,943.114766,19.051987,10.191065,5.784747,0.5,4.205345
10,80.000000,86.444099,6.444099,41.526407,72.091837,95.170168,23.078331,532.609351,804301,18Х3МФБ,...,0.14483,0.00712,0.0,0.839606,874.330783,18.300459,9.617827,5.396623,0.5,3.583364
10 исторический,79.974490,NaN,NaN,NaN,73.979592,NaN,NaN,NaN,803320,18ХМФБ,...,0.16000,0.00900,NaN,0.541333,863.982622,18.860207,11.734794,2.298203,0.5,1.838345
100,79.000000,105.275544,26.275544,690.404224,62.500000,100.784530,38.284530,1465.705276,801072,37Г2Ф,...,0.00006,0.00770,0.0,0.670405,836.231880,18.723616,10.394012,12.956744,0.2,2.361924
100 исторический,77.244898,NaN,NaN,NaN,65.943878,NaN,NaN,NaN,803335,30ХГМА,...,0.15000,0.00700,NaN,0.632333,875.055055,18.655439,10.725832,25.304650,0.5,2.990870
1000,77.000000,105.275544,28.275544,799.506401,67.000000,100.784530,33.784530,1141.394501,802143,37Г2Ф,...,0.00006,0.00770,0.0,0.670405,829.051634,18.655439,10.405192,12.818432,0.2,2.361924
1000 исторический,75.000000,NaN,NaN,NaN,65.500000,NaN,NaN,NaN,803400,37Г2Ф,...,0.00000,0.00400,NaN,0.692000,832.690230,18.886918,10.489792,10.333368,0.2,2.637781


In [9]:
a = pd.DataFrame()
a
b = pd.DataFrame({'a':[0]})
pd.concat([a, b])

,a
0,0


In [ ]:
# df_dict = pd.read_excel('DATA/modes_before_model/top_typesize(рекомендации).xlsx', sheetname=None)
# for sheet, df in zip(df_dict, df_dict.values()):
#     valid = clean_data_sheets(df, ls_need_col+['№ партии', 'марка стали'])
#     print(valid)

In [32]:
y_valid = tmp[targets[0]]
y_pred = tmp['predicted '+targets[0]]

from sklearn.metrics import r2_score 
from sklearn.metrics import explained_variance_score 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_absolute_error
print('corr',np.corrcoef(y_valid, y_pred)[0][1])
print('r2', r2_score(y_valid, y_pred))
print('explained_variance_score', explained_variance_score(y_valid, y_pred))
print('mean_squared_error', mean_squared_error(y_valid, y_pred))
print('mean_absolute_error', mean_absolute_error(y_valid, y_pred))
print('median_absolute_error', median_absolute_error(y_valid, y_pred))

d = {
    'corr': [np.corrcoef(y_valid, y_pred)[0][1]],
    'r2': [r2_score(y_valid, y_pred)],
    'explained_variance_score': [explained_variance_score(y_valid, y_pred)],
    'mean_squared_error': [mean_squared_error(y_valid, y_pred)],
    'mean_absolute_error': [mean_absolute_error(y_valid, y_pred)],
    'median_absolute_error': [median_absolute_error(y_valid, y_pred)]
}

pd.DataFrame(d).to_excel(output_filename+'_scores.xlsx')

corr 0.9491442637311868
r2 0.8930216852957163
explained_variance_score 0.8957239874637852
mean_squared_error 19.292150409068473
mean_absolute_error 3.2350262905404064
median_absolute_error 2.4166564587036916


In [31]:
y_valid = tmp[targets[1]]
y_pred = tmp['predicted '+targets[1]]

from sklearn.metrics import r2_score 
from sklearn.metrics import explained_variance_score 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_absolute_error
print('corr',np.corrcoef(y_valid, y_pred)[0][1])
print('r2', r2_score(y_valid, y_pred))
print('explained_variance_score', explained_variance_score(y_valid, y_pred))
print('mean_squared_error', mean_squared_error(y_valid, y_pred))
print('mean_absolute_error', mean_absolute_error(y_valid, y_pred))
print('median_absolute_error', median_absolute_error(y_valid, y_pred))

d = {
    'corr': [np.corrcoef(y_valid, y_pred)[0][1]],
    'r2': [r2_score(y_valid, y_pred)],
    'explained_variance_score': [explained_variance_score(y_valid, y_pred)],
    'mean_squared_error': [mean_squared_error(y_valid, y_pred)],
    'mean_absolute_error': [mean_absolute_error(y_valid, y_pred)],
    'median_absolute_error': [median_absolute_error(y_valid, y_pred)]
}

pd.DataFrame(d).to_excel(output_filename+'_scores.xlsx')

corr 0.9643158026543087
r2 0.9213201363358962
explained_variance_score 0.9219271770797535
mean_squared_error 11.392219003303019
mean_absolute_error 2.49501703374625
median_absolute_error 1.9771326530612328


In [93]:
for target in targets:
    print(
        df[df['MAE '+target]>5].shape[0]/df.shape[0],
          df[df['MAE '+target]>4].shape[0]/df.shape[0],
        df[df['MAE '+target]>3].shape[0]/df.shape[0])

0.20423000660938534 0.29610046265697293 0.4137475214805023
0.11103767349636484 0.18968935888962327 0.29808327825512226


In [17]:
d ={
   '% err >5': [df[df['MAE']>5].shape[0]/df.shape[0]],
   '% err >4':  [df[df['MAE']>4].shape[0]/df.shape[0]],
   '% err >3':  [df[df['MAE']>3].shape[0]/df.shape[0]]
}

pd.DataFrame(d).to_excel(dir_name+'/%.xlsx')

In [18]:
d

{'% err >3': [0.4527112232030265],
 '% err >4': [0.33921815889029006],
 '% err >5': [0.25598991172761665]}